In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)

In [ ]:
# とりあえずデータ読み込め！
# 選挙結果のデータ
pres08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/pres08.csv')
# 世論調査のデータ
polls08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/polls08.csv')
# それぞれのデータ内に、オバマとマケインの得票率(もしくは支持率)の差を「margin」カラムとして保存
pres08['margin'] = pres08['Obama'] - pres08['McCain']
polls08['margin'] = polls08['Obama'] - polls08['McCain']

# 同じ州の同じ中間日に世論調査が行われているのか？を確認
polls08.groupby(['state','middate']).count()
# 行われてるので、平均をとる必要がある

# pythonでのDate型ってこうやるらしい
polls08['middate'] = pd.to_datetime(polls08['middate'])
# middate は最大でも20081101
polls08['middate'].max()

# 試しに色々計算させてみる
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11')
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16')
import math
abs(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16'))
type(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11'))

# 選挙日までの日数をカラムとして保存
polls08['DaysToElection'] = pd.to_datetime('2008-11-04') - polls08['middate']

# DaysToElectionを小さい方から並べたランクナンバーを振る
polls08 = polls08.assign(
    rankMin=polls08.groupby(by='state')['DaysToElection'].rank(method='min')
)
# 最も選挙日に近い世論調査に限定して、オバマとマケインの差の平均をとり、それを予測値カラム(カラム名：pred)として保存
poll = polls08[polls08['rankMin']==1].groupby(by='state', as_index=False).mean()[['state','margin']]
poll = poll.rename(columns={'margin':'pred'})
# 州の名前をつけ直す
poll = pd.merge(poll, pres08, how='left', on='state')
# 誤差カラムを追加
poll['error'] = poll['margin'] - poll['pred']
# 誤差の平均を算出(＝平均予測誤差)
poll['error'].mean()
# root mean squre(2乗平均平方根)で誤差を見てみる
math.sqrt((poll['error']**2).mean())

In [ ]:
myfig, ax1 =plt.subplots(ncols=1, nrows=1, figsize=(8,6))
ax1 = sns.distplot(poll['error'], ax=ax1, bins=[-20, -15, -10, -5, 0, 5, 10, 15, 20])
ax1.axvline(poll['error'].mean())
ax1.text(x=poll['error'].mean(), y=0.06, s='平均誤差')
myfig.show()

世論調査結果と、実際の選挙結果の関係性をビジュアルで把握するため、x軸に世論調査結果、y軸に選挙結果をプロットする

In [ ]:
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
# 一旦グラフを描かせるために、アルファゼロでプロットしちゃう
ax1 = sns.scatterplot(x='pred', y='margin', data=poll, alpha=0)
# プロットの位置に、州名を文字としておいていく
for index, row in poll.iterrows():
    print(row['pred'], row['margin'], row['state'])
    ax1.text(x=row['pred'], y=row['margin'], s=row['state'], ha='center', va='center')
# 45度線を描く
ax1 = sns.lineplot(x=[-40, 80], y=[-40, 80], ax=ax1, color='gray')
# seaborn経由で線グラフを破線にしたい場合、これ以外に方法ないんだろうか・・
ax1.lines[0].set_linestyle('--')
ax1.axvline(0, linestyle="-", color='gray')
ax1.axhline(0, color='gray')
myfig.show()

今回の選挙結果予測においては、各候補者の支持率差分を選挙結果(得票率)の予測値として使用したが、細かな値の差はさほど重要ではなく、大事なのは、「オバマ勝利と予測して、実際本当にオバマが勝利したかどうか？」といった、「勝った」「負けた」の判断を正しく予測できたか？になる。
そこで、今回の世論調査が、選挙結果を予測するものとして間違っていた州を把握してみる。

In [ ]:
# 予測値、結果、ともに、正負の符号が一致していれば、勝敗の予測が正しかったことになるので、
# 符号が異なっていた州名をリストアップしてみる
poll[(poll['pred'] > 0) != (poll['margin'] > 0)]

どの州も、支持率の差が1％ポイント程度しかない接戦だったことがわかる。

世論調査が、オバマへの選挙人表の数を予測できていたのかを確認してみる。

In [ ]:
# その州の選挙人数は、得票率が高い候補者に投票するので、以下のような計算で算出できる
poll[poll['margin'] > 0]['EV'].sum()

# 世論調査での選挙人数
poll[poll['pred'] > 0]['EV'].sum()

実際の結果は364人、世論調査では349人と、15人の差が出ているが、勝利の境目が270人であるので、「勝つ」「負ける」の予測としては、世論調査も正しく予測できていた。

実際の得票率を世論調査の支持率で予測できるかどうか(＝世論調査は正しいのかどうか？)と、世論調査の変遷を確かめる方法を考えてみる。
pollsUS08.csvには、選挙期間中の世論調査のデータが入っている。この時系列のデータを用いて、以下のような方法での検証を試みる。

- 当該日から7日前までの移動平均値を当該日の値として算出
- 世論調査が正しく集められているのであれば、選挙当日までの世論の変遷は時系列グラフで読み解くことができ、かつ、最終的に投票日の選挙結果と全日の世論調査の支持率の結果がニアリーになっているはず。

In [ ]:
# 何はともあれデータ読み込み
pollsUS08 = pd.read_csv('officialSource/qss-master/PREDICTION/pollsUS08.csv')
pollsUS08.dtypes
# 日付文字列をpandas.datetime64に変換
pollsUS08['middate'] = pd.to_datetime(pollsUS08['middate'])
pollsUS08.dtypes

# 選挙前90日間のデータを作っていく
# 一旦、何日分のデータがあるのかを確認
pollsUS08["middate"].describe()
# 選挙日は2008-11-04 なので、前日までのデータが入っていることを確認。

# 日付演算、「1日前」「3ヶ月後」などは、下記のように timedelta型のインスタンスで計算させる。
pollsUS08.middate[0] - pd.to_timedelta(1,'day')
from dateutil.relativedelta import relativedelta
# ○ヶ月後、みたいな処理をする場合、月末日を考慮してもらうためには、dateutil.relativedelta を使うと良い
pollsUS08.middate[0] + relativedelta(months=3)
pd.to_datetime('2019-02-28') - relativedelta(months=1)
# 下記のような計算の場合、ちゃんと考慮して2月の末日を返してくれる。SQLのADD_MONTHS関数的なやつ
pd.to_datetime('2019-01-31') + relativedelta(months=1)
pd.to_datetime('2019-01-31') + relativedelta(days=3)

# 結論、「○日後」「○ヶ月前」みたいな計算は、dateutil.relativedelta 使っとけ、って感じですかね。

# 本論に戻りまして。
# pollsUS08のうち、選挙日2008-11-04 から90日前までのデータに限定して、日毎に支持率の平均を計算していく。
temp = pollsUS08[pollsUS08['middate'] > (pd.to_datetime('2008-11-04') - relativedelta(days=90))].copy()

# あ。。。SQLのウィンドウ関数的なんがわからんので、検証。
temp.rolling('5D', on='middate').mean()
# んーー、ウィンドウ関数を判断する対象となるカラムは、重複データがあると動かないらしい。まあ、そうか・・。

# しょうがないので、一旦日別のデータで下準備する
temp2 = temp.groupby(by='middate', as_index=False).sum()
temp2 = temp2.rename(columns={'McCain': 'SUM_McCain', 'Obama':'SUM_Obama'})
temp2 = temp2.merge(temp.groupby(by='middate', as_index=False).count()[['middate','Pollster']], how='left', on='middate')
temp2 = temp2.rename(columns={'Pollster':'count'})
# からの、7日間の移動平均。本では当該日を含まない7日前のデータで平均を取ってるけど、
# ちょっとめんどくさいし、普通の移動平均チックな考え方だと当該日含んでるので、当該日を含む7日前の
# 平均値を取る
temp3 = temp2.rolling('7D', on='middate').sum()
temp3 = temp3.rename(columns={'McCain': 'SUM_McCain', 'Obama':'SUM_Obama'})
temp3['MEAN_McCain'] = temp3['SUM_McCain']/ temp3['count']
temp3['MEAN_Obama'] = temp3['SUM_Obama']/ temp3['count']
# 選挙結果のレコードを挿入
s = pd.Series([pd.to_datetime('2008-11-04'), np.nan, np.nan, 1, 45.65, 52.93], index=temp3.columns)
temp3 = temp3.append(s, ignore_index=True)
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax1 = sns.lineplot(data=temp3, x='middate', y='MEAN_McCain', label='McCain', ax=ax1)
ax1 = sns.lineplot(data=temp3, x='middate', y='MEAN_Obama', label='Obama', ax=ax1)
myfig.show()

In [ ]:
from sklearn import linear_model, datasets
model = linear_model.LinearRegression()
# 説明変数のリストは、データフレーム型でないと通らない。sklearnは説明変数が1つの前提で組まれていないらしい。
model.fit(face['d.comp'].to_frame(), face['diff_share'])
model.coef_
model.intercept_

仮説：1秒未満での顔の見た目で、民主党の候補者のほうが能力が高いと答えた割合で、民主党候補の選挙結果が予測できる

この仮説を確かめる前に、民主党候補者の見た目スコアと、実際の得票率の差分の分布の関係を可視化してみる

In [ ]:
# まず読み込み
face = pd.read_csv('officialSource/qss-master/PREDICTION/face.csv')
# 得票率の差分を出す
face['d_share'] = face['d.votes'] / (face['d.votes'] + face['r.votes'])
face['r_share'] = face['r.votes'] / (face['d.votes'] + face['r.votes'])
face['diff_share'] = face['d_share'] - face['r_share']
# 描画していく
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax1 = sns.scatterplot(data=face, x='d.comp', y='diff_share', hue='w.party', ax=ax1)
ax1.set_xlim([0, 1])
ax1.set_ylim([-1, 1])
ax1.xaxis.label.set_text('民主党候補の見た目能力スコア')
ax1.yaxis.label.set_text('得票率における民主党候補のマージン')
ax1.title.set_text('見た目の能力と得票率')
myfig.show()

In [ ]:
# 相関係数を出す 手法その1
# pandas の相関係数は、メソッドが選べて、各列総当たり戦をしてくれる。
# Null除去もリストワイズ的ではなく、ちゃんと使う列でNullがあった時のみ、除去としてくれている。
face[["d.comp", "diff_share"]].corr()

# 相関係数を出す 手法その2 scipy で出すと帰無仮説(無相関)のp値も出してくれる
import scipy.stats
scipy.stats.pearsonr(face["d.comp"], face['diff_share'])
# 相関係数が0.4327743... 、p値がすっごい小さい。
# 弱い相関はありそうね。


In [ ]:
from sklearn import linear_model, datasets
model = linear_model.LinearRegression()
# 説明変数のリストは、データフレーム型でないと通らない。sklearnは説明変数が1つの前提で組まれていないらしい。
model.fit(face['d.comp'].to_frame(), face['diff_share'])
model.coef_
model.intercept_

In [ ]:
# 実際にこのモデルを使って予測値を出してみる。
model.predict(face['d.comp'].to_frame())

# 実際の値と、線形モデルをプロットしてみる。
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
ax1 = sns.scatterplot(data=face, x='d.comp', y='diff_share', ax=ax1, color='gray')
ax1 = sns.lineplot(x=face['d.comp'], y=model.predict(face['d.comp'].to_frame()), ax=ax1)
# サンプル通り、x軸y軸に平均線を入れてみる
ax1.axvline(face['d.comp'].mean())
ax1.axhline(face['diff_share'].mean())
ax1.lines[1].set_linestyle('--')
ax1.lines[2].set_linestyle('--')
myfig.show()
myfig.savefig('ch4/image1.png')

In [ ]:
# RMSE で評価してみる
from sklearn.metrics import mean_squared_error
mes = mean_squared_error(face['diff_share'], model.predict(face['d.comp'].to_frame()))
np.sqrt(mes)

### 4.2.5 Rにおけるデータの結合
オバマ大統領の2008年の得票率と、2012年の得票率を用い知恵、平均への回帰現象が見られるかどうかを検証していく。

In [ ]:
# 2012年のデータを読みこむ
pres12 = pd.read_csv('officialSource/qss-master/PREDICTION/pres12.csv')

# 前後の値どちらもある事が前提になるので、Inner Join しちゃう
pres08_pres12 = pd.merge(pres08, pres12, how='inner', on="state", suffixes=['_08', '_12'])

# オバマさんの得票率をz score 化しちゃう
# scipy 使うと便利っぽい
# Zスコア化するときに標準偏差や分散算出時の自由度をどうするのかはキッチリ考えようね。普通は不偏で考える？ので？自由度1を指定しておく。
import scipy
pres08_pres12[['Obama_08_z', 'Obama_12_z']] = pres08_pres12[['Obama_08', 'Obama_12']].apply(scipy.stats.zscore, ddof=1)

# 得られたz得点で、回帰モデルを作ってみる
model_obama = linear_model.LinearRegression()
model_obama.fit(pres08_pres12[['Obama_08_z']], pres08_pres12['Obama_12_z'])
model_obama.coef_
model_obama.intercept_

In [ ]:
# 得られたモデルの情報と、実際の観察データをプロット
myfig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax = sns.scatterplot(x=pres08_pres12['Obama_08_z'], y=pres08_pres12['Obama_12_z'], color='gray', ax=ax)
# モデルに実際の観測値xを食わせて、予測値yを出してプロットさせる
ax = sns.lineplot(x=pres08_pres12['Obama_08_z'], y=model.predict(pres08_pres12['Obama_08_z'].to_frame()), ax=ax)
ax.set_xlim(-4.2)
ax.set_ylim(-4.2)
myfig.show()

In [ ]:
# 2008年の得票率の下位25％のグループを作る
under25_2008 = pres08_pres12[(pres08_pres12['Obama_08_z'] <= pres08_pres12['Obama_08_z'].quantile(0.25))].copy()
# 2008年under25%のうち、2012年の得票率の方が高い州の割合
((under25_2008['Obama_12_z']) > (under25_2008['Obama_08_z'])).mean()

# 2008年の上位75％のグループを作る
upper75_2008 = pres08_pres12[pres08_pres12['Obama_08_z'] >= pres08_pres12['Obama_08_z'].quantile(0.75)].copy()
# そのうち、2012年の得票率が高い週の割合
(upper75_2008['Obama_12_z'] > upper75_2008['Obama_08_z']).mean()

結果としては、2008年のオバマ得票率下位25％では、2012年の方が得票率が高い州は57％。
一方で、上位75％では、2012年の方が得票率が高い州は46％。

つまり、2008年に高い得票率を得た州の2012年の得票率は、2008年を上回りにくい＝平均の回帰が見られる。

### モデルの当てはまり
フロリダ州における、1996年の投票結果を用いて、2000年の投票結果を予測するモデルを作成し、その当てはまり度合いを決定係数で見てみる.
本来は、1996年のAさんの投票結果データを使って学習させ、同じくAさんの2000年の投票結果を予測させるのが理想だが、実際にはそこまで都合よく同じ候補者が2回連続で立候補してないので、「同じ政党のAさんとBさん」が

In [ ]:
# 読み込み
florida = pd.read_csv("officialSource/qss-master/PREDICTION/florida.csv")
# fit
model_florida = linear_model.LinearRegression()
model_florida.fit(florida['Perot96'].to_frame(), florida['Buchanan00'])
model_florida.intercept_, model_florida.coef_
# 決定係数を見る専用の関数があるらしい
import sklearn.metrics as metrics
metrics.r2_score(florida['Buchanan00'], model_florida.predict(florida['Perot96'].to_frame()))
# 一応、定義通りに算出もしてみる
import math
TSS = (florida['Buchanan00'] - florida['Buchanan00'].mean()).apply(lambda x:math.pow(x, 2)).sum()
SSR = (model_florida.predict(florida['Perot96'].to_frame()) - florida['Buchanan00']).apply(lambda x: math.pow(x, 2)).sum()
(TSS - SSR) / TSS 

この決定係数 0.5103 が高いのか低いのかを考えるための比較対象として、オバマの州レベルでの得票率が2008年と2012年で相関していたことを発見した調査結果を利用し、その時の回帰分析のR2を比較してみる。

In [ ]:
metrics.r2_score(pres08_pres12['Obama_12_z'], model_obama.predict(pres08_pres12[['Obama_08_z']]))

フロリダの分析での決定係数の値が割と低そうであることは確認できた。
その要因がなんなのか？を可視化をするために、予測値と残差をスキャッタープロットしてみる

In [ ]:
x = model_florida.predict(florida['Perot96'].to_frame())
y = florida['Buchanan00'] - x
myfig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8,6))
ax = sns.scatterplot(x=x, y=y, ax=ax, color='gray')
ax.axhline()
ax.xaxis.label.set_text('当てはめ値')
ax.yaxis.label.set_text('残差')
myfig.show()

残差が2000と大きくハズレている箇所があるので、どういう観察れコートで外れているのかを確認する。

In [ ]:
florida[y == y.max()]

大きくハズレているのはパームビーチ郡であることがわかった。
なぜパームビーチ郡でこのような外れ値(ブキャナンにやたらと票が入っている)状況になったかというと、パームビーチ郡では __チョウ型投票用紙__ が使われたが、この用紙が非常に紛らわしく、当時人気のあったアル・ゴアの票が誤ってブキャナンに流れてしまったことが要因と言われている。
(ちなみに、当時アル・ゴアはブッシュよりも50万票多く獲得していたが、フロリダ州で537票差でブッシュが勝利したため、ブッシュが最終的に大統領になった)

では、イレギュラーな状況だったパームビーチ郡のデータを除外すると、モデルの当てはまりは改善するのか？を確かめてみる

In [ ]:
# まずは、パームビーチ郡を除いて学習させて、同じく残差プロットを見てみる
florida_omitPB = florida[florida['county']!='PalmBeach'].copy()
model_omitPB = linear_model.LinearRegression()
model_omitPB.fit(florida_omitPB['Perot96'].to_frame(), florida_omitPB['Buchanan00'])
x = model_omitPB.predict(florida_omitPB['Perot96'].to_frame())
y = florida_omitPB['Buchanan00'] - x
myfig2, ax2 = plt.subplots(nrows=1,ncols=1, figsize=(8,6))
ax2 = sns.scatterplot(x=x, y=y, color='gray', ax=ax2)
ax2.set_ylim(-600, 3000)
ax2.set_xlim(0, 1500)
ax2.axhline(0)
ax2.title.set_text('パームビーチ郡を除いた残差プロット')
ax2.xaxis.label.set_text('当てはめ値')
ax2.yaxis.label.set_text('残差')
myfig2.show()

In [ ]:
# パームビーチの有無別でモデルを作成し、その当てはまり度合いを可視化してみる

myfig3, ax3 = plt.subplots(ncols=1, nrows=1, figsize=(8,6))
pred = model_florida.predict(florida_omitPB['Perot96'].to_frame())
pred_omitPB = model_omitPB.predict(florida_omitPB['Perot96'].to_frame())
ax3 = sns.scatterplot(florida['Perot96'], florida['Buchanan00'], ax=ax3, color='gray')
ax3 = sns.lineplot(x=florida_omitPB['Perot96'], y=pred, label='全州', ax=ax3)
ax3 = sns.lineplot(x=florida_omitPB['Perot96'], y=pred_omitPB, label='PB除外', ax=ax3)
ax3.legend()
myfig3.show()

## 回帰分析と因果関係
ここまで見てきた回帰分析が、物事の因果関係を見抜く際にどう役立つのかを、実際の分析を通して見ていく。

### ランダム化実験
女性政治家の存在が実施される政策の内容に与える因果効果を検証した研究を例に見てみる。

この場合、観察データとして「女性政治家が居る村と居ない村で、実施された内容別の政策実施回数の差を見る」という事が考えられるが、「リベラルな選挙区では女性政治家が通りやすい」「リベラルな選挙区で実施される政策と、そうでない選挙区での政策との間には、内容の偏りがある」という事も想定される。
つまり、交絡因子が存在する事になる(交絡因子＝リベラル or 保守)ので、ランダム化実験が必要。

今回用意されているデータは、ランダム化実験でデザインされていて、ランダム抽出された村に対して「かならず女性村議長を選べ」というお達しを出し、その村における政策内容を観察した。

In [ ]:
women = pd.read_csv("officialSource/qss-master/PREDICTION/women.csv")
# 女性の議長を置けと言われた村が、ちゃんと女性議長をおいているかどうか、念の為確認
# reserved の村の女性議員フラグはすべて1のハズ＝平均が1
women[women['reserved'] == 1].mean()['female']
women[women['reserved'] == 0].mean()['female']

女性議員は女性市民の意向を実現しやすいという仮定を置いてみる。
女性市民は飲用水についての政策実施を望んでおり、男性は灌漑についての政策実施を望んでいるというデータが他分析で明らかになっているので、「女性議員が居る村＝飲用水政策の実施をしがち」という仮説を立て、検証してみる。

In [ ]:
# 飲用水政策の実施回数の平均因果効果の推定
women.loc[women['reserved'] == 1, 'water'].mean() - women.loc[women['reserved'] == 0, 'water'].mean()

# 灌漑政策の実施回数の平均因果効果の推定
women.loc[women['reserved'] == 1, 'irrigation'].mean() - women.loc[women['reserved'] == 0, 'irrigation'].mean()

女性議長が居る村の飲用水政策の実施回数は、平均で9回くらい多い。
女性議長が居る村の灌漑政策の実施回数は、とくに大きな差はない。

ここまでの平均因果効果の推定結果と回帰モデルとの関係性を見ていく

In [ ]:
# トリートメントグループである(＝1)、トリートメントグループではない(＝0)を説明変数とした回帰モデルを想定してモデルを作ってみる。
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(women['reserved'].to_frame(), women['water'])
model.coef_, model.intercept_
# 一応可視化
fig1, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax1 = sns.scatterplot(x=women['reserved'], y=women['water'], ax=ax1, color='gray')
ax1 = sns.lineplot(x=women['reserved'], y=model.predict(women['reserved'].to_frame()), ax=ax1)
fig1.show()
fig1.savefig('ch4/飲用水政策実施回数の違い.png')

model2 = linear_model.LinearRegression()
model2.fit(women['reserved'].to_frame(), women['irrigation'])
model2.coef_, model2.intercept_
# 一応可視化
fig2, ax2 = plt.subplots(ncols=1, nrows=1, figsize=(8,6))
ax2 = sns.scatterplot(x=women['reserved'], y=women['irrigation'], ax=ax2, color='gray')
ax2 = sns.lineplot(x=women['reserved'], y=model2.predict(women['reserved'].to_frame()), ax=ax2)
fig2.show()
fig2.savefig('ch4/灌漑政策実施回数の違い.png')